In [1]:
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

set_config(display='diagram')

In [2]:
df = pd.DataFrame(np.random.randint(0,1000,size=(10000, 4)), columns=list('ABCD'))
df.head()

,A,B,C,D
0,755,948,788,364
1,530,214,536,397
2,79,436,58,330
3,736,89,920,26
4,49,340,57,932


In [3]:
df1 = pd.DataFrame()
for idx1,col1 in enumerate(df.columns):
    for idx2,col2 in enumerate(df.drop(columns=df.columns[:idx1]).columns):
        df1[col1+'x'+col2] = df.loc[:,col1] * df.loc[:,col2]

df1.head()

,AxA,AxB,AxC,AxD,BxB,BxC,BxD,CxC,CxD,DxD
0,570025,715740,594940,274820,898704,747024,345072,620944,286832,132496
1,280900,113420,284080,210410,45796,114704,84958,287296,212792,157609
2,6241,34444,4582,26070,190096,25288,143880,3364,19140,108900
3,541696,65504,677120,19136,7921,81880,2314,846400,23920,676
4,2401,16660,2793,45668,115600,19380,316880,3249,53124,868624


In [4]:
df2 = df.merge(df1, left_index=True, right_index=True)

df2.head()

,A,B,C,D,AxA,AxB,AxC,AxD,BxB,BxC,BxD,CxC,CxD,DxD
0,755,948,788,364,570025,715740,594940,274820,898704,747024,345072,620944,286832,132496
1,530,214,536,397,280900,113420,284080,210410,45796,114704,84958,287296,212792,157609
2,79,436,58,330,6241,34444,4582,26070,190096,25288,143880,3364,19140,108900
3,736,89,920,26,541696,65504,677120,19136,7921,81880,2314,846400,23920,676
4,49,340,57,932,2401,16660,2793,45668,115600,19380,316880,3249,53124,868624


In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
norm_scal = StandardScaler()

In [7]:
df3 = pd.DataFrame(index=df2.index, columns=df2.columns)

In [8]:
for col in df2.columns:
    df3[col] = norm_scal.fit_transform(df2[col].values.reshape(-1,1))

In [9]:
df3.head()

,A,B,C,D,AxA,AxB,AxC,AxD,BxB,BxC,BxD,CxC,CxD,DxD
0,0.871354,1.539998,1.013701,-0.475045,0.780807,2.074419,1.571010,0.096039,1.880247,2.246063,0.422893,0.984923,0.177586,-0.675447
1,0.097133,-1.000501,0.144408,-0.361014,-0.184793,-0.625760,0.162509,-0.189062,-0.972482,-0.606603,-0.752361,-0.137652,-0.158191,-0.591777
2,-1.454749,-0.232121,-1.504488,-0.592531,-1.102080,-0.979806,-1.103892,-1.005014,-0.489840,-1.009996,-0.486138,-1.092955,-1.036416,-0.754062
3,0.805975,-1.433147,1.469045,-1.642999,0.686196,-0.840565,1.943366,-1.035706,-1.099163,-0.754686,-1.125765,1.743481,-1.014739,-1.114636
4,-1.557978,-0.564393,-1.507937,1.487671,-1.114905,-1.059531,-1.111998,-0.918267,-0.739008,-1.036650,0.295515,-1.093342,-0.882296,1.777138


In [10]:
df3.shape[1]

14

In [11]:
from sklearn.decomposition import PCA

In [12]:
pca = PCA(n_components=df3.shape[1])

In [13]:
pca.fit_transform(df3)

array([[ 3.39391332,  2.46557766, -1.62779726, ..., -0.22417927,
         0.12985542,  0.07761826],
       [-1.35265993, -0.18145862,  1.00877827, ..., -0.11967224,
        -0.09125284, -0.15450927],
       [-3.41052611, -0.11721703, -0.70427328, ...,  0.21465685,
         0.41524234,  0.23797298],
       ...,
       [-0.67560659,  0.97584749, -2.22226443, ...,  0.08465935,
        -0.19696135, -0.09884207],
       [ 1.43610553, -1.15318529, -3.98359452, ..., -0.21929073,
         0.03184544,  0.01273689],
       [ 3.46392304,  2.2749638 ,  0.92806032, ...,  0.08216389,
        -0.19495706,  0.26189016]])

In [14]:
vec_var = pca.explained_variance_

In [15]:
pca_comp = (vec_var>1).sum()

In [16]:
pca2 = PCA(n_components=pca_comp)

In [17]:
mat4 = pca2.fit_transform(df3)

In [18]:
mat4.shape

(10000, 4)

In [19]:
Z = df['A'] + df['C']

In [20]:
Z.head()

0    1543
1    1066
2     137
3    1656
4     106
dtype: int32

In [21]:
columns = ['col'+str(i) for i in range(mat4.shape[1]+1)]

In [22]:
df5 = pd.DataFrame(index=df1.index, columns=columns)

In [23]:
df5.loc[:,df5.columns[:-1]] = mat4
df5.loc[:,df5.columns[-1]]  = Z

In [24]:
df5.head()

,col0,col1,col2,col3,col4
0,3.39391,2.46558,-1.6278,-1.14291,1543
1,-1.35266,-0.181459,1.00878,-0.925217,1066
2,-3.41053,-0.117217,-0.704273,0.914964,137
3,-0.999053,1.55006,2.02364,-3.35509,1656
4,-2.233,-1.93111,0.263968,2.67258,106


# NN con dati originali

In [25]:
import torch

In [26]:
df.head()

,A,B,C,D
0,755,948,788,364
1,530,214,536,397
2,79,436,58,330
3,736,89,920,26
4,49,340,57,932


In [27]:
df['Z'] = Z

In [28]:
df.head()

,A,B,C,D,Z
0,755,948,788,364,1543
1,530,214,536,397,1066
2,79,436,58,330,137
3,736,89,920,26,1656
4,49,340,57,932,106


In [29]:
in_data = df

In [30]:
in_data_tens = torch.from_numpy(in_data.values)

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
in_data_tens_train, in_data_tens_cv = train_test_split(in_data, test_size=0.20)

In [33]:
in_data

,A,B,C,D,Z
0,755,948,788,364,1543
1,530,214,536,397,1066
2,79,436,58,330,137
3,736,89,920,26,1656
4,49,340,57,932,106
...,...,...,...,...,...
9995,534,508,559,108,1093
9996,335,322,921,120,1256
9997,671,744,354,108,1025
9998,753,986,110,572,863


In [34]:
x_train = in_data_tens_train[:,:-1]
y_train = in_data_tens_train[:,-1]
x_cv = in_data_tens_cv[:,:-1]
y_cv = in_data_tens_cv[:,-1]

x_train.shape,y_train.shape,x_cv.shape,y_cv.shape

TypeError: '(slice(None, None, None), slice(None, -1, None))' is an invalid key

In [ ]:
y_train

In [ ]:
x_train.shape[1]

In [ ]:
import torch
from torch import nn

In [37]:
nn_model = nn.Sequential(
    nn.Linear(x_train.shape[1],250),
    nn.ReLU(),
    nn.Linear(250,250),
    nn.ReLU(),
    nn.Linear(250,250),
    nn.ReLU(),
    nn.Linear(250,250),
    nn.ReLU(),
    nn.Linear(250,1),
    nn.ReLU(),
)

In [38]:
nn_model(x_cv).sum()

tensor(11498.1699, grad_fn=<SumBackward0>)

In [39]:
error = nn.MSELoss()

In [40]:
learning_rate = 0.001
optimizer = torch.optim.SGD(nn_model.parameters(), lr = learning_rate)

In [176]:
for i in range(1000):
    y_pred = nn_model(x_train) #calcolo l'uscita
    loss = error(y_pred, y_train.reshape(-1,1))  #calcolo l'errore
    loss.backward()        #calcolo del gradiente
    optimizer.step()       #aggiornamento dei parametri
    optimizer.zero_grad()  #azzeramento del gradiente
    if np.mod(i,100)==0:
        print(loss.data)

tensor(1144302.2500)
tensor(nan)


KeyboardInterrupt: 

In [ ]:
y_pred.shape, y_train.reshape(-1,1).shape

In [ ]:
error(y_pred, y_train)

In [ ]:
for param in nn_model.parameters():
    print(param)

In [35]:
import inspect
import math
from sklearn import (
    base, compose, decomposition, linear_model,
    metrics, model_selection, multioutput,
    neural_network, pipeline, preprocessing,
    svm, utils
)

In [36]:
class TorchMLPClassifier(base.BaseEstimator, base.ClassifierMixin):

    """
    Adapted from kaggle.com/graymant/pytorch-regression-with-sklearn-pipelines
    """
    def __init__(
        self, output_dim=1, hidden_layer_dims=[100,],
        num_epochs=1, learning_rate=0.01, batch_size=128, shuffle=False,
        callbacks=[], use_gpu=False, verbose=1, use_bn=False, batches_per_epoch=None,
    ):
        self._history = None
        self._model = None
        if use_gpu and not torch.cuda.is_available():
            raise RuntimeError('GPU was requested but is not available.')
        
        self._gpu = use_gpu

        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        for arg, val in values.items():
            setattr(self, arg, val)

    def _build_model(self, input_dim):
        self._layer_dims = [input_dim] + \
            self.hidden_layer_dims + [self.output_dim]

        self._model = torch.nn.Sequential()
        for idx, dim in enumerate(self._layer_dims):
            if (idx < len(self._layer_dims) - 1):
                module = torch.nn.Linear(dim, self._layer_dims[idx + 1])
                torch.nn.init.xavier_uniform_(module.weight)
                self._model.add_module("linear" + str(idx), module)

            if (idx < len(self._layer_dims) - 2):
                if self.use_bn:
                    self._model.add_module("bn" + str(idx), torch.nn.BatchNorm1d(self._layer_dims[idx + 1]))
                self._model.add_module("relu" + str(idx), torch.nn.ReLU())

        if self._gpu:
            self._model = self._model.cuda()

    def _train_model(self, X, y):
        torch_x = torch.from_numpy(X).float()
        torch_y = torch.from_numpy(y).float()
        if self._gpu:
            torch_x = torch_x.cuda()
            torch_y = torch_y.cuda()

        train = torch.utils.data.TensorDataset(torch_x, torch_y)
        train_loader = torch.utils.data.DataLoader(
            train, batch_size=self.batch_size, shuffle=self.shuffle
        )

        loss_fn = torch.nn.BCEWithLogitsLoss()
#         loss_fn = torch.nn.BCELoss()

        optimizer = torch.optim.Adam(
            self._model.parameters(), lr=self.learning_rate
        )

        self._history = {"loss": [], "val_loss": [], "log_loss": []}

        def gen_():
            if self.batches_per_epoch is None:
                while True:
                    yield from enumerate(train_loader)
                    yield None
            else:
                iidx = 0
                it = itertools.chain.from_iterable(iter(lambda:enumerate(train_loader), None))
                while True:
                    yield next(it)
                    iidx += 1
                    if not idx % self.batches_per_epoch:
                        yield None
                    
        train_batches = gen_()
        
        finish = False
        for epoch in range(self.num_epochs):
            if finish:
                break

            loss = None
            idx = 0
            for batch in train_batches:
                if batch is None:
                    break

                idx, (minibatch, target) = batch
                y_pred = self._model(torch.autograd.Variable(minibatch))

                loss = loss_fn(
                    y_pred,
                    torch.autograd.Variable(target.float())
                )

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                print(loss)

            y_labels = target.cpu().numpy() if self._gpu else target.numpy()
            y_pred_results = y_pred.cpu().data.numpy() if self._gpu else y_pred.data.numpy()

            loss_val = loss.data.cpu().numpy().item() if self._gpu else loss.data.numpy().item()
            self._history["log_loss"].append(loss_val)

            if self.verbose > 0:
                print(
                    "Results for epoch {}, log_loss {}".format(
                        epoch + 1, loss_val
                    )
                )
            for callback in self.callbacks:
                callback.call(self._model, self._history)
                if callback.finish:
                    finish = True
                    break

    def fit(self, X, y, reset = True):
        assert (type(self.output_dim) == int), "output_dim must be defined"
        
        if reset:
            self._build_model(*X.shape[1:])
        self._train_model(X, y)

        return self

    def predict_proba(self, X, y=None):
        return 1/(1 + np.exp(-self.predict_log_proba(X, y)))
    
    def predict_log_proba(self, X, y=None):
        if self._history == None:
            raise RuntimeError("Regressor has not been fit")

        results = []
        split_size = math.ceil(len(X) / self.batch_size)

        for batch in np.array_split(X, split_size):
            x_pred = torch.autograd.Variable(torch.from_numpy(batch).float())
            y_pred = self._model(x_pred.cuda() if self._gpu else x_pred)
            y_pred_formatted = y_pred.cpu().data.numpy() if self._gpu else y_pred.data.numpy()
            results.append(y_pred_formatted)

        results = np.concatenate(results).astype(np.float)
        return results


In [37]:
nn = TorchMLPClassifier(hidden_layer_dims=[1024], use_bn=True, output_dim=1)

In [38]:
x = df.loc[:,df.columns[:-1]].values
y = df.loc[:,df.columns[-1]].values

In [39]:
y.shape

(10000,)

In [40]:
nn.batch_size = 128
nn.num_epochs = 100
nn.hidden_layer_dims = [2048]

In [41]:
nn.fit(x, y.reshape(-1,1))

tensor(-579.0176, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-11801.7246, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-22349.3848, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-34954.9258, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-45841.9453, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-58757.3438, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-65713.9453, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-79219.0156, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-95127.4062, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-107185.6406, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-126127.6719, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-139995.8594, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-158444.2031, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-180894.9219, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-194731.4844, grad_fn=<BinaryCrossEntr

KeyboardInterrupt: 

In [92]:
nn.predict_proba(x)

array([[1.        ],
       [0.82693206],
       [1.        ],
       ...,
       [1.        ],
       [1.        ],
       [0.82693206]])

In [115]:
y_pred = nn._model(torch.autograd.Variable(torch.from_numpy(x).type(torch.float32)))

In [116]:
y_pred

tensor([[2.8170e+07],
        [7.8927e+01],
        [3.1063e+07],
        ...,
        [7.4568e+06],
        [4.1472e+07],
        [7.8927e+01]], grad_fn=<AddmmBackward>)

In [117]:
loss_fn = torch.nn.BCEWithLogitsLoss()

In [118]:
loss = loss_fn(y_pred, y)

TypeError: 'int' object is not callable

In [103]:
df

,A,B,C,D,Z
0,561,474,580,970,1141
1,162,776,369,20,531
2,990,921,199,982,1189
3,913,964,629,422,1542
4,313,354,800,205,1113
...,...,...,...,...,...
9995,118,143,527,884,645
9996,115,206,354,877,469
9997,669,816,63,41,732
9998,863,319,538,156,1401
